In [1]:
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import torch.optim as optim
from torchvision import transforms
import torchvision.models as models

from sklearn.model_selection import train_test_split

from PIL import Image

In [2]:
torch.manual_seed(42)

In [3]:
df = pd.read_csv("/home/krish/Datasets/fashion_mnist/fashion-mnist_train.csv")
df.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,0,0,0,0,0,0,0,5,0,...,0,0,0,30,43,0,0,0,0,0
3,0,0,0,0,1,2,0,0,0,0,...,3,0,0,0,0,1,0,0,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
X = df.iloc[:, 1:].values
y = df.iloc[:, 0].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [16]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [6]:
custom_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [17]:
class customDataset(Dataset):

    def __init__(self, features, labels, transforms):
        super().__init__()
        self.features = features
        self.labels = labels
        self.transforms = transforms

    def __len__(self):
        return len(self.features)

    def __getitem__(self, index):
        image = self.features[index].reshape(28,28)
        image = image.astype(np.uint8)
        image = np.stack([image]*3, axis=-1)
        image = Image.fromarray(image)

        image = self.transforms(image)

        return image, torch.tensor(self.labels[index], dtype=torch.long)

In [19]:
train_dataset = customDataset(X_train, y_train, transforms=custom_transforms)
test_dataset = customDataset(X_test, y_test, transforms=custom_transforms)

In [34]:
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=True)

In [41]:
vgg16 = models.vgg16(pretrained=True)

In [42]:
for param in vgg16.features.parameters():
    param.requires_grad = False

In [43]:
vgg16.classifier = nn.Sequential(
    nn.Linear(25088, 1024),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(1024, 512),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(512, 10)
)

vgg16 = vgg16.to(device)

In [44]:
epochs = 50
learning_rate = 0.001

In [45]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(vgg16.classifier.parameters(), lr=learning_rate)

In [46]:
for epoch in range(epochs):

    Epoch_loss = 0
    for batch_features, batch_labels in train_loader:

        batch_features = batch_features.to(device)
        batch_labels = batch_labels.to(device)

        output = vgg16(batch_features)

        loss = criterion(output, batch_labels)

        optimizer.zero_grad()
        loss.backward()

        optimizer.step()
        Epoch_loss += loss.item()

    avg_epoch_loss = Epoch_loss/len(batch_features)
    print(f"Loss after Epoch {epoch+1}: {avg_epoch_loss}")

Loss after Epoch 1: 4.787210360984318
Loss after Epoch 2: 3.299162086797878
Loss after Epoch 3: 2.9281619300018065
Loss after Epoch 4: 2.6499542325909715
Loss after Epoch 5: 2.420607581763761
Loss after Epoch 6: 2.257636032780283
Loss after Epoch 7: 2.066087337108911
Loss after Epoch 8: 2.0417206734709907
Loss after Epoch 9: 1.945690448192181
Loss after Epoch 10: 1.7812143445808033
Loss after Epoch 11: 1.653055662247425
Loss after Epoch 12: 1.5484869726460602


KeyboardInterrupt: 